<a href="https://colab.research.google.com/github/carlosjviola/MeLi-challenge/blob/main/MeLi_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**MeLi** - Data Visualization & Analytics Challenge

Carlos José Viola - 2022/11 

[Ver especificaciones](https://drive.google.com/file/d/1CTPIC5OIZ58joVDMG9AYAEecKTCsRtlc/view?usp=share_link)

Supuestos / Consideraciones

1) El archivo tiene registros correspondientes al mes de enero de 2022 para el campo first_login_dt

2) No hay especificaciones para el campo week_year. Se considera corresponde a la semana del año del campo first_login_dt

3) Se realizan distintas operaciones de limpieza de datos

3) Los registros del 1 y 2 de enero vienen con la semana 52. De acuerdo a lo supuesto en el punto anterior se reemplazarán los valores.

4) Removemos 

In [2]:

import pandas as pd
import numpy as np
from datetime import date

# Se crea una variable para almacenar nombre del archivo a analizar
archivoCsv = 'https://raw.githubusercontent.com/carlosjviola/MeLi-challenge/main/dataset_challenge_analytics.csv'
# Cargamos el archivo en un DataFrame de Pandas realizar el estudio
dfUsrOrig = pd.read_csv(archivoCsv)

#Vemos el tamaño del archivo (filas, columnas) y los primeros 5 registros del mismo
print("Cantidad de filas y columnas")
print(dfUsrOrig.shape)

print("\nMuestra las primeras 10 filas")
print(dfUsrOrig.head(10))

print("\nMuestra las últimas 10 filas")
print(dfUsrOrig.head(10))

Cantidad de filas y columnas
(11500, 12)

Muestra las primeras 10 filas
   Unnamed: 0 first_login_dt  week_year         user_id  habito habito_dt  \
0           0     2022-01-30          4   MLB2883855850     0.0       NaN   
1           1     2022-01-30          4   MLB7059024930     0.0       NaN   
2           2     2022-01-07          1   MLB4163269880     0.0       NaN   
3           3     2022-01-07          1   MLB7349806870     0.0       NaN   
4           4     2022-01-07          1   MLB7239064190     0.0       NaN   
5           5     2022-01-01         52  MLB10318536730     0.0       NaN   
6           6     2022-01-01         52   MLB7546352300     0.0       NaN   
7           7     2022-01-16          2   MLB2785023500     0.0       NaN   
8           8     2022-01-16          2   MLB8395954300     0.0       NaN   
9           9     2022-01-16          2   MLB1610695500     0.0       NaN   

   activacion activacion_dt  setup    setup_dt  return   return_dt  
0          

In [ ]:
#Removemos las primera columna que contiene un id que entiendo no vamos a utilizar
dfUsr = dfUsrOrig.drop(columns = dfUsrOrig.columns[0], axis = 1, inplace= False)

print("El nuevo DataFrame tiene una columna menos")
print(dfUsr.shape)


El nuevo DataFrame tiene una columna menos
(11500, 11)


In [ ]:
#Vamos a analizar la columna user_id y ordenamos esa columna
dfUsr.sort_values('user_id', inplace= True)
print(dfUsr['user_id'].head(10))
print(dfUsr['user_id'].tail(10))

print(dfUsr['user_id'].sample(n=100))

5561    MLB10001480800
9631    MLB10001530370
5035    MLB10002487550
8479    MLB10002855700
3189    MLB10003112290
3103    MLB10003554750
7609    MLB10003698160
2770    MLB10004252630
4929    MLB10004759110
2343    MLB10005576530
Name: user_id, dtype: object
10269    MLB988455940-1
3478       MLB988498130
3896       MLB990480920
2104       MLB991448540
2393       MLB991879190
7881       MLB995538640
1206       MLB995826480
11206      MLB995826480
8786       MLB997458170
4300       MLB999142060
Name: user_id, dtype: object
9442        MLB2092484360
5988       MLB10531203970
10822     MLB2683679600-1
7318        MLB7762051130
2276        MLB2341408400
               ...       
6530       MLB10655605650
10416     MLB2458144930-1
10745    MLB10517994170-1
220         MLB2794998380
2088        MLB7270346260
Name: user_id, Length: 100, dtype: object


In [ ]:
#Reemplazamos los caracteres '-1' que aparecen al final de algunas líneas
dfUsr['user_id'] = dfUsr['user_id'].str.replace('-1','')

#Vamos a verificar que el campo user_id tenga valores únicos
print("Cantidad total registros")
print(len(dfUsr['user_id']))

print("\Candidad registros únicos")
print( len(dfUsr['user_id'].unique()) )

#Cómo han aparecido duplicados vamos a eliminarlos.
#Para esto creamos una serie que identifique los duplicados
lSerie = dfUsr['user_id'].duplicated(keep = False) 
 
dfUsr = dfUsr[~lSerie] 

print("Nueva cantidad total registros")
print(len(dfUsr['user_id']))



Cantidad total registros
8500
\Candidad registros únicos
8500
Nueva cantidad total registros
8500


In [ ]:
# Convertimos la columna hábito en entero de acuerdo a las especificaciones
# antes llenamos con 0 los valores nulos en esta columna
dfUsr['habito'] = dfUsr['habito'].fillna(0.0).astype(int)
  
# Verificamos los cambios
display(dfUsr.dtypes)

first_login_dt    object
week_year          int64
user_id           object
habito             int64
habito_dt         object
activacion         int64
activacion_dt     object
setup              int64
setup_dt          object
return             int64
return_dt         object
dtype: object

In [ ]:
# Convertimos en date los datos date-time en las columnas fecha. Haciendo una inspección rápida del csv se observa que no hay horas
dfUsr['first_login_dt'] = pd.to_datetime(dfUsr['first_login_dt']).dt.date
dfUsr['habito_dt'] = pd.to_datetime(dfUsr['habito_dt']).dt.date
dfUsr['activacion_dt'] = pd.to_datetime(dfUsr['activacion_dt']).dt.date
dfUsr['setup_dt'] = pd.to_datetime(dfUsr['setup_dt']).dt.date
dfUsr['return_dt'] = pd.to_datetime(dfUsr['return_dt']).dt.date


In [ ]:
# Como no está muy claro su contenido vamos a analizar la columna week_year
dfUsr['week_year'].agg(['mean', 'max', 'min'])

mean     4.348471
max     52.000000
min      1.000000
Name: week_year, dtype: float64

In [ ]:
#Los datos son de enero, no puede haber registros con el valor 52 en la semana
print(dfUsr[dfUsr['week_year']==52])

     first_login_dt  week_year         user_id  habito habito_dt  activacion  \
2770     2022-01-02         52  MLB10004252630       0       NaT           1   
9577     2022-01-01         52  MLB10019564580       0       NaT           0   
5717     2022-01-01         52  MLB10064714530       0       NaT           0   
3170     2022-01-01         52  MLB10074070400       0       NaT           0   
4404     2022-01-02         52  MLB10088145650       0       NaT           1   
...             ...        ...             ...     ...       ...         ...   
4280     2022-01-02         52   MLB8381923230       0       NaT           1   
7115     2022-01-01         52    MLB852124930       0       NaT           1   
5591     2022-01-02         52    MLB938961270       0       NaT           1   
4761     2022-01-01         52    MLB949054360       0       NaT           1   
5733     2022-01-02         52    MLB950258540       0       NaT           0   

     activacion_dt  setup    setup_dt  

In [ ]:
#dfError = dfUsr[ dfUsr['activacion_dt'] < dfUsr['first_login_dt']]
def calc_new_col(row):
    columnas = ['activacion_dt', 'setup_dt', 'habito_dt', 'return_dt']
    max_dt = date.min 
    for col in columnas:
        if row[col] > max_dt:
            max_dt = row[col]
    return max_dt
    #return max(row['col1'], row['col2'], row['col3'])

dfUsr["max_dt"] = dfUsr.apply(calc_new_col, axis=1)
print(dfUsr.head()) 
print(dfUsr.shape) 

# printing the dataframe
#print(dfUsr)
#print(dfUsr.count())
#print(dfUsr['return_dt'].notnull().sum())


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: Comparison of NaT with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.
  


    first_login_dt  week_year        user_id  habito habito_dt  activacion  \
30      2022-01-15          2   MLB722272670       0       NaT           1   
47      2022-01-12          2   MLB980528780       0       NaT           1   
82      2022-01-14          2  MLB1603055200       0       NaT           1   
112     2022-01-27          4  MLB2412718380       0       NaT           1   
132     2022-01-16          2  MLB2014154210       0       NaT           1   

    activacion_dt  setup setup_dt  return   return_dt  
30     2022-01-12      0      NaT       0         NaT  
47     2022-01-11      0      NaT       0         NaT  
82     2022-01-13      0      NaT       1  2022-01-20  
112    2022-01-26      0      NaT       0         NaT  
132    2022-01-13      0      NaT       1  2022-01-17  
(430, 11)


In [ ]:

indexUsr = dfUsr[ dfUsr['max_dt'] < dfUsr['first_login_dt']].index
dfUsrFinal = dfUsr.drop(indexUsr)
print(dfUsrFinal.head(15))
print(dfUsrFinal.shape)
dfUsr.to_csv('resultado2.csv')

   first_login_dt  week_year         user_id  habito   habito_dt  activacion  \
0      2022-01-30          4   MLB2883855850       0         NaT           1   
1      2022-01-30          4   MLB7059024930       0         NaT           1   
2      2022-01-07          1   MLB4163269880       0         NaT           0   
3      2022-01-07          1   MLB7349806870       0         NaT           0   
4      2022-01-07          1   MLB7239064190       0         NaT           1   
6      2022-01-01         52   MLB7546352300       0         NaT           0   
7      2022-01-16          2   MLB2785023500       0         NaT           1   
8      2022-01-16          2   MLB8395954300       0         NaT           0   
9      2022-01-16          2   MLB1610695500       0         NaT           1   
10     2022-01-16          2   MLB3148290290       0         NaT           0   
11     2022-01-16          2   MLB1633460060       0         NaT           1   
14     2022-01-23          3   MLB509964

# Nueva sección